<a href="https://colab.research.google.com/github/Zulkazeem/CLIP-AI-Pavement-Image-Classification/blob/main/Using_Roboflow_and_CLIP_for_Pavement_Surface_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Install CLIP Dependencies

In [ ]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess
!pip install torch

import numpy as np
import torch
import os

In [ ]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 164 (delta 31), reused 49 (delta 19), pack-reused 91
Receiving objects: 100% (164/164), 8.87 MiB | 27.51 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/CLIP


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="xvSbc97nneJxnVuWwVx7")
project = rf.workspace("pavement-project").project("pavement-project")
dataset = project.version(1).download("clip")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Pavement-Project-1 in clip:: 100%|██████████| 958/958 [00:00<00:00, 5352.11it/s]


In [ ]:
dataset.location

'/content/Pavement-Project-1'

In [ ]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir(dataset.location + '/test/')
class_names.remove('_tokenization.txt')
class_names

['Cracked Non-Cracked',
 'Spalling',
 'Cracked',
 'Joint Non-Cracked',
 'Non-Cracked']

In [ ]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat {dataset.location}/test/_tokenization.txt

An example picture from the Pavement Project dataset depicting a Cracked
An example picture from the Pavement Project dataset depicting a Cracked, and a Non Cracked
An example picture from the Pavement Project dataset depicting a Joint, and a Non Cracked
An example picture from the Pavement Project dataset depicting a Non Cracked
An example picture from the Pavement Project dataset depicting a Spalling

In [ ]:
#edit your prompts as you see fit here, be sure the classes are in teh same order as above
%%writefile {dataset.location}/test/_tokenization.txt
An example picture from the Pavement Project dataset depicting a Cracked
An example picture from the Pavement Project dataset depicting a Non Cracked
An example picture from the Pavement Project dataset depicting a Spalling

Overwriting /content/Pavement-Project-1/test/_tokenization.txt


In [ ]:
candidate_captions = []
with open(dataset.location + '/test/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

# Run CLIP inference on your classification dataset

In [ ]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=d828eb39fd1f3e21615f3b803979c61ca2ee8179855fce8d40ed41de9602118f
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [ ]:
import torch
from torchvision import datasets, transforms
from transformers import CLIPProcessor, CLIPModel
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

correct = []

In [ ]:
#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
#text = clip.tokenize(candidate_captions).to(device)
text = processor(candidate_captions, return_tensors="pt", padding=True, truncation=True).to(device)

In [ ]:
!pip install torchvision
!pip install transformers --upgrade

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust size based on CLIP model requirements
    transforms.ToTensor(),
])


\Downloads\pavement_1	est


In [ ]:
for cls in class_names:
    class_correct = []
    test_imgs = glob.glob(dataset.location + '/test/' + cls + '/*.jpg')
    for img in test_imgs:
        #print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            inputs = processor(text, images=Image.open( test_imgs), return_tensors="pt", padding=True, truncation=True)
            outputs = model(**inputs)

            image_features = model.encode_image(image)
            text_features = model.encode_text(text)

            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            #print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)

    print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
print('accuracy on all is : ' + str(sum(correct)/len(correct)))


In [ ]:
from torchvision import transforms
from PIL import Image
import torch
import glob

# Assuming you have already loaded the CLIP model and processor
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# Assuming class_names is a list of class names in your dataset

# Define the data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

correct = []

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob(dataset.location + '/test/' + cls + '/*.jpg')

    for img in test_imgs:
        image = transform(Image.open(img)).unsqueeze(0).to(device)

        # Use processor to tokenize text
        text = ["a photo of", "a picture of"]
        inputs = processor(text, images=image, return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad():
            # Directly use the encoded features for classification
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits_per_image[0], dim=-1).cpu().numpy()

            pred = class_names[probs.argmax()]

            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)

    print(f'Accuracy on class {cls} is: {sum(class_correct)/len(class_correct)}')

print(f'Accuracy on all classes is: {sum(correct)/len(correct)}')

Accuracy on class Cracked Non-Cracked is: 0.0
Accuracy on class Spalling is: 1.0
Accuracy on class Cracked is: 0.0
Accuracy on class Joint Non-Cracked is: 0.0
Accuracy on class Non-Cracked is: 0.0
Accuracy on all classes is: 0.3225806451612903
